In [1]:
import re
import requests
from bs4 import BeautifulSoup
import psycopg2
from psycopg2 import sql
import time




In [ ]:
DATABASE_URL = 'postgres://u7r3q8fi3oil8d:p7c4ab7923edcd720fad4e7373c00c69924b36cd5449778bcf0e5a70268616cdd@cbec45869p4jbu.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d2jp5l4pcu7iuv'
conn    = psycopg2.connect(DATABASE_URL, sslmode="require")
cursor  = conn.cursor()

In [10]:
cursor.close()
conn.close()

In [119]:
TELE_GROUPS = {
    # "index_n_bonkbot_present": -1002022757177, # -100: search exactly GetHistoryRequest, while positive number is to get cached of a user
    "index_n_gmgn_present": -1002202241417
}

print(f"{list(TELE_GROUPS.keys())}")

def clean_special_characters(value):
    # Remove or escape special characters
    if isinstance(value, str):
        # Example: Removing single quotes and other special characters
        value = re.sub(r"[\'\"\\]", "", value)
    return str(value)

['index_n_gmgn_present']


In [120]:
# token_info = [{'network':'Solana','ca':'ca123pump', 'ticker':'ticket01'},{'network':'Solana','ca':'ca456pump','ticker':'ticker02'}]

token_info = [{'network': 'test', 'ca': 'cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', 'ticker': 'trump', 'token_name': 'trump2024', 'description': 'trump2024', 'market_cap': 93919.63, 'liquidity_pool_value': 179235.06, 'total_holder': 48, 'gmgn_url': 'https://gmgn.ai/sol/token/CPnkXXMwX38E3pB81aDwewoFsvXyGqLbTwwev7HneQ2t', 'index_n_smartwallet_bought': 0, 'index_n_gmgn_present': 0, 'index_n_bonkbot_present': 1, 'index_n_follower_twitter': None, 'index_n_subscriber_telegram': 0, 'index_n_twitter_search_result': None, 'twitter_url': None, 'telegram_url': None, 'website_url': None}]

# ---
def update_token_pool_summary_tbl(conn, cursor, token_info):
    """The function is to update the token_pool_summary table
    Args:
        token_info (list): _description_
    """
    try:
        # Prepare the insert statement dynamically
        start_time = time.time()
        queries = ""
        cols = [f"p_{item}" for item in list(token_info[0].keys())]

        # Apply SCD Type 2 for every single token, and insert into token_pool_summary table
        for row in token_info:
            # Clean data input
            values = [clean_special_characters(val) for val in row.values()]
            values = list(map(":=".join, zip(cols, values)))
            values = (str(values).replace(":=", ":='").replace(", '", ", ").replace("['", '').replace(']', '')).replace('"', '')

            # Prepare insert statement
            scd2_statement = ("""
                SELECT update_scd_to_token_pool_info_summary({parameters});
            """).format(
                parameters = values
            )
            queries += scd2_statement + "\n"

        # Execute
        try:
            cursor.execute(queries)
        except (psycopg2.Error) as e:
            print(f"Function `update_token_pool_summary_tbl` | Error inserting row {row}: {e}")
            conn.rollwback() # Rollback the current transaction to skip the faulty row
        else:
            # Commit transaction
            conn.commit()

        print(f"{len(token_info)} records have been added to `token_pool_info_summary`.")

        # Elapse time & return
        elapsed_time = round(time.time() - start_time, 2)
        print(f"*** 5. Update `token_pool_summary` table | Elapsed time: {elapsed_time}sec")

    except (Exception) as error:
        print(f"Update `token_pool_summary_tbl` error occured with message {error}") 



update_token_pool_summary_tbl(conn, cursor, token_info)

1 records have been added to `token_pool_info_summary`.
*** 5. Update `token_pool_summary` table | Elapsed time: 0.87sec


In [114]:
token_info = [{'network': 'Solana', 'ca': 'cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', 'ticker': 'trump', 'token_name': 'trump2024', 'description': 'trump2024', 'market_cap': 93919.63, 'liquidity_pool_value': 179235.06, 'total_holder': 48, 'gmgn_url': 'https://gmgn.ai/sol/token/CPnkXXMwX38E3pB81aDwewoFsvXyGqLbTwwev7HneQ2t', 'index_n_smartwallet_bought': 0, 'index_n_gmgn_present': 0, 'index_n_bonkbot_present': 1, 'index_n_follower_twitter': None, 'index_n_subscriber_telegram': 0, 'index_n_twitter_search_result': None, 'twitter_url': None, 'telegram_url': None, 'website_url': None}]

# Need to convert it to : [(p_network:='Solana', p_ca:='ca123pump'), (p_network:='Solana', p_ca:='ca456pump')]
def clean_special_characters(value):
    # Remove or escape special characters
    if isinstance(value, str):
        # Example: Removing single quotes and other special characters
        value = re.sub(r"[\'\"\\]", "", value)
    return str(value)

cols = [f"p_{item}" for item in list(token_info[0].keys())]
values = []

for row in token_info:
    values = [clean_special_characters(val) for val in row.values()]
    print(f"1: {values}")
    values = list(map(":=".join, zip(cols, values)))
    print(f"2: {values}")
    values = (str(values).replace(":=", ":='").replace(", '", ", ").replace("['", '').replace(']', '')).replace('"', '')
    print(f"3: {values}")
    

1: ['Solana', 'cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', 'trump', 'trump2024', 'trump2024', '93919.63', '179235.06', '48', 'https://gmgn.ai/sol/token/CPnkXXMwX38E3pB81aDwewoFsvXyGqLbTwwev7HneQ2t', '0', '0', '1', 'None', '0', 'None', 'None', 'None', 'None']
2: ['p_network:=Solana', 'p_ca:=cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', 'p_ticker:=trump', 'p_token_name:=trump2024', 'p_description:=trump2024', 'p_market_cap:=93919.63', 'p_liquidity_pool_value:=179235.06', 'p_total_holder:=48', 'p_gmgn_url:=https://gmgn.ai/sol/token/CPnkXXMwX38E3pB81aDwewoFsvXyGqLbTwwev7HneQ2t', 'p_index_n_smartwallet_bought:=0', 'p_index_n_gmgn_present:=0', 'p_index_n_bonkbot_present:=1', 'p_index_n_follower_twitter:=None', 'p_index_n_subscriber_telegram:=0', 'p_index_n_twitter_search_result:=None', 'p_twitter_url:=None', 'p_telegram_url:=None', 'p_website_url:=None']
3: p_network:='Solana', p_ca:='cpnkxxmwx38e3pb81adwewofsvxygqlbtwwev7hneq2t', p_ticker:='trump', p_token_name:='trump2024', p_descripti

In [121]:
from sqlalchemy import create_engine, Column, Integer, String, LargeBinary
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import os

DATABASE_URL = 'postgres://u7r3q8fi3oil8d:p7c4ab7923edcd720fad4e7373c00c69924b36cd5449778bcf0e5a70268616cdd@cbec45869p4jbu.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d2jp5l4pcu7iuv'
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()


NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:postgres

In [ ]:

class File(Base):
    __tablename__ = 'files'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    content = Column(LargeBinary)

Base.metadata.create_all(engine)

def upload_file(file_name):
    with open(file_name, 'rb') as f:
        content = f.read()
    new_file = File(name=file_name, content=content)
    session.add(new_file)
    session.commit()

def download_file(file_name, output_name=None):
    if output_name is None:
        output_name = file_name
    file = session.query(File).filter_by(name=file_name).first()
    with open(output_name, 'wb') as f:
        f.write(file.content)


In [2]:
import re
import requests
from bs4 import BeautifulSoup
import psycopg2
from psycopg2 import sql
import time
import sqlite3


In [ ]:
conn = sqlite3.connect('./tele_session_file_login_private.session')


In [2]:
from telethon import TelegramClient
from create_tele_session_file import TELE_SESSION_FILE_LOGIN, TELE_SESSION_FILE_PATH, V_API_ID, V_API_HASH
from create_tele_session_file import download_binary_file


client = TelegramClient(TELE_SESSION_FILE_LOGIN, V_API_ID, V_API_HASH)

async def check_tele_conn_auth():
    print("Run check_tele_conn_auth...")
    await client.connect()
    
    if client.is_connected():
        print("TelegramClient connected!")
    else:
        print("TelegramClient not connected!")
    
    if client.is_user_authorized():
        print("TelegramClient is authorized!")
    else:
        print("TelegramClient not authorized!")

    await client.disconnect()

In [5]:
client.loop.run_until_complete(check_tele_conn_auth())


<coroutine object check_tele_conn_auth at 0x7fa1818b5460>

In [4]:
# Fetch the content of the URL
url = 'https://t.me/baneCTO'
response = requests.get(url)
if response.status_code != 200:
    print(f"Crawl_keyword function | Failed to fetch the URL: {url}")
    # return None

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')
body_text = soup.get_text().lower()
# Search for number of subscribers
match = re.search(r'(\d[\d\s]*)\s+[subscribers,members]', body_text)

if match:
    subscribers = int(match.group(1).replace(" ",""))
    if subscribers:
        print(subscribers)
    
    

1804


In [1]:
import requests

ALCHEMY_API_TOKEN  = "8lkJ9fP65TFrEXHYY_M6eoc_cxjGZ1IX"


In [8]:
import requests

def get_solana_token_holders_quicknode(quicknode_url, token_mint_address):
    headers = {"Content-Type": "application/json"}

    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getProgramAccounts",
        "params": [
            "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA",  # This is the program ID for the SPL Token Program
            {
                "encoding": "jsonParsed",
                "filters": [
                    {"dataSize": 165},  # Data size for token accounts
                    {
                        "memcmp": {
                            "offset": 0,  # Starting point of the filter
                            "bytes": token_mint_address  # The token mint address to filter by
                        }
                    }
                ]
            }
        ]
    }

    response = requests.post(quicknode_url, json=payload, headers=headers)
    if response.status_code == 200:
        result = response.json()
        accounts = result.get("result", [])
        print(f"accounts: {accounts}")
        unique_holders = {account['account']['data']['parsed']['info']['owner'] for account in accounts}
        return len(unique_holders)
    else:
        print("Error:", response.status_code, response.text)
        return None

# Example usage
quicknode_url = "https://greatest-serene-frog.solana-mainnet.quiknode.pro/d9956a657fc3ca286b0ee009465a5b704753fb95/"  # Replace with your QuickNode endpoint URL
token_mint_address = "0x3885fbe4CD8aeD7b7e9625923927Fa1CE30662A3"  # Replace with your token mint address
holders_count = get_solana_token_holders_quicknode(quicknode_url, token_mint_address)
if holders_count is not None:
    print(f"Number of unique wallet holders: {holders_count}")


[]
Number of unique wallet holders: 0


In [12]:
from web3 import Web3

def get_token_addresses_from_pair(web3, pair_address):
    # Uniswap V2 Pair ABI
    pair_abi = [
        {
            "constant": True,
            "inputs": [],
            "name": "token0",
            "outputs": [{"internalType": "address", "name": "", "type": "address"}],
            "payable": False,
            "stateMutability": "view",
            "type": "function"
        },
        {
            "constant": True,
            "inputs": [],
            "name": "token1",
            "outputs": [{"internalType": "address", "name": "", "type": "address"}],
            "payable": False,
            "stateMutability": "view",
            "type": "function"
        }
    ]

    pair_contract = web3.eth.contract(address=pair_address, abi=pair_abi)

    # Get token addresses
    token0 = pair_contract.functions.token0().call()
    token1 = pair_contract.functions.token1().call()

    return token0, token1

# Example usage
alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/8lkJ9fP65TFrEXHYY_M6eoc_cxjGZ1IX"  # Replace with your Alchemy API key
web3 = Web3(Web3.HTTPProvider(alchemy_url))

pair_address = "0x3885fbe4CD8aeD7b7e9625923927Fa1CE30662A3"  # Replace with the pair address you have
token0, token1 = get_token_addresses_from_pair(web3, pair_address)

print(f"Token 0 Address: {token0}")
print(f"Token 1 Address (Mint Address): {token1}")


Token 0 Address: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
Token 1 Address (Mint Address): 0xEE2a03Aa6Dacf51C18679C516ad5283d8E7C2637


In [12]:
text = '$Wallet_Tracking_SOL$🟢 BUY KAMONISM  on PUMP FUN\n5zkcswATYbHasqiYGC6b2921URRtDF9cuLsYDDmMXfbg (Exp_Day_Trader_05)\n\n🔹Exp_Day_Trader_05 swapped 1 SOL for 4,259,137.63 ($144.84) KAMONISM  @$0.0000340\n✊Holds: 15,823,149.6 KAMONISM  (1.58%)\n\n🔹Exp_Day_Trader_05:\nSOL: -1.02 ($-147.59)\nKAMONISM : +4,259,137.63 ($144.84)\n\n🔗 KAMONISM  | MC: $34.01K | Seen: 10m: BE | DS | DT | PH | Bullx | Pump\nCVFKvW6ygukVytgb3yikn2WrDHgPj5FMJ8sbbeqPEb2n'
balance_change_pattern = r"\n([A-Za-z0-9 ]+): ([+-]?[0-9.,]+) \(\$([+-]?[0-9,.]+)\)"


balance_changes = {}
for match in re.finditer(balance_change_pattern, text):
    currency = match.group(1)
    amount = float(match.group(2).replace(',', ''))
    usd_value = float(match.group(3).replace(',', ''))
    print(f"Currency: {currency}, Amt: {amount}, USD: {usd_value}")
    if(currency != 'SOL' and currency != 'USDT' and currency != 'USDC'):
        balance_changes["ticker"] = currency
        balance_changes["ticker_amt"] = amount
        balance_changes["ticker_usd_val"] = usd_value

        print(balance_changes)




Currency: SOL, Amt: -1.02, USD: -147.59
Currency: KAMONISM , Amt: 4259137.63, USD: 144.84
{'ticker': 'KAMONISM ', 'ticker_amt': 4259137.63, 'ticker_usd_val': 144.84}


In [23]:
import re

def extract_keywords(text):
    extracted_data = {}
    # Regular expressions to extract values
    patterns = {
        "fdv": r"FDV:\s*🏛\s*\$([\d,.]+[KM]?)",
        "liquidity": r"Liquidity:\s*💧\s*\$([\d,.]+[KM]?)",
        "age": r"Age:\s*🌿\s*([\d\w]+)",
        "24h_txns_total": r"24H Txns:\s*🔁\s*Total:\s*([\d,.KM]+)",
        "ca": r"📄 CA:\s*([\w\d]+)",
        "price_chg_24h": r"Price Chg:\s*📈\s*24H:\s*🟢\s*([\d,.%]+)",
        "24h_makers_buyers":  r"24H Makers:\s*🥸\s*Total:.*?\n\s*Buyers:\s*([\d,.KM]+)",
        "24h_makers_sellers": r"24H Makers:\s*🥸\s*Total:.*?\n\s*Buyers:.*?\n\s*Sellers:\s*([\d,.KM]+)",
        "now_trending_at": r"is now trending at\s*([\d️⃣]+)"
    }
    # Extract values using regex
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            extracted_data[key] = match.group(1)
        else:
            extracted_data[key] = None
    # Convert "Now trending at" value to a numeric format
    if "Now trending at" in extracted_data:
        numeric_value = re.sub(r"\D", "", extracted_data["Now trending at"])  # Remove non-digit characters
        extracted_data["Now trending at"] = int(numeric_value) if numeric_value.isdigit() else None
    # Respond
    return extracted_data


In [24]:

# Example usage
text = """$NewPool_SOL_DexScreener$✨ Token info updated for HEIL ELON (HEIL ELON) (https://dexscreener.com/solana/8o74Z9xB4qGSbQGg2Wp9pAwDc1Kod7ETw2YM8jCMzBHM)!


      Price: 🏷️ $0.0001285
        FDV: 🏛️ $128.5K
  Liquidity: 💧 $40.9K (103.43 SOL)
        Age: 🌿 3m

     Volume: 💰
        24H: $58.5K
         6H: $58.5K
         1H: $58.5K
         5M: $58.5K

  Price Chg: 📈
        24H: 🟢 59.93%
         6H: 🟢 59.93%
         1H: 🟢 59.93%
         5M: 🟢 59.93%

   24H Txns: 🔁
      Total: 569
       Buys: 391
      Sells: 178

 24H Makers: 🥸
      Total: 384
     Buyers: 345
    Sellers: 164


ℹ️ Info:
HEIL ELON
🔗Twitter (𝕏) (https://x.com/kanyewest/status/1888336448673481193)  

📄 CA: ykPcmFuaN3JhRsbBxu83t3QN7w9EDFqJiutQ8mKpump"""

# Run extraction
result = extract_keywords(text)
result

{'fdv': None,
 'liquidity': '40.9K',
 'age': '3m',
 '24h_txns_total': '569',
 'ca': 'ykPcmFuaN3JhRsbBxu83t3QN7w9EDFqJiutQ8mKpump',
 'price_chg_24h': '59.93%',
 '24h_makers_buyers': '345',
 '24h_makers_sellers': '164',
 'now_trending_at': None}

In [32]:
import requests
import datetime

# 🔑 Replace with your Twitter API Bearer Token
API_KEY = "KNHEMSzwxu2oaf1csMbzYMyA2"
API_KEY_SECRET = "zG5qT9DNRmkIkGxygxJgcxBxVjB6XM5zB6T8FDdD4Apm3VGp39"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAC3XywEAAAAA8cV6esyLq9N6G0AdlpPLEFeZYzk%3DzU21JssItcovAGnE9524im1RvnUPiCW21NHvtOUqaQxx8SNIxh"

def search_tweets(keyword, min_replies=50, max_results=10):
    """Search top tweets containing `keyword` with at least `min_replies` in the last 24 hours."""
    
    # Get yesterday's date (24 hours ago)
    since_date = (datetime.datetime.utcnow() - datetime.timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
    
    # Twitter API endpoint
    url = "https://api.twitter.com/2/tweets/search/recent"
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}", "User-Agent": "v2RecentSearchPython"}
    
    # Query: Search keyword, min 50 replies, last 24h, no retweets, English
    query = f'{keyword} -is:retweet min_replies:{min_replies} lang:en'

    params = {
        "query": query,
        "tweet.fields": "created_at,public_metrics,author_id",
        "max_results": max_results,
        "sort_order": "relevancy"
    }
    # Make API request
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

# 🔍 Run search with the given keyword
keyword = "7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump"
tweets = search_tweets(keyword)
tweets

'{"errors":[{"parameters":{"query":["7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump -is:retweet min_replies:50 lang:en"]},"message":"There were errors processing your request: Reference to invalid field \'min_replies\' (at position 58), Reference to invalid operator \'min_replies\'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 58)"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}'

In [45]:
# Sample text
text = """$NewPool_SOL_GmgnKolFomo$  88 KOL Buy CAR (https://gmgn.ai/sol/token/7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump)!
🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢

💵 KOL Inflow净流入:$638.4K(3153.6645 Sol) 
💳 KOL Buy/Sell:441/383

$CAR(Central African Republic Meme)
7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump

📈 5m | 1h | 6h: 5.53% | -17.23% | -38.11%
🎲 5m TXs/Vol: 687/$334.3K
💡 MCP: $123.8M
💧 Liq: 5,555.24 SOL ($2.2M 🔥100%)
👥 Holder: 34878
🕒 Open: 17h ago

✅ NoMint / ✅Blacklist / ✅Burnt
✅TOP 10: 7.03%

⏳ DEV: 🚨 Sell All (https://gmgn.ai/sol/address/121ftnYRm3WJmDHCWrazzRoyZAzZQ4xc5XD7dp3sfpfo)
👨‍🍳 DEV Burnt烧币: 0(🔥Rate: %)

Backup BOT: US (https://t.me/US_GMGNBOT?start=7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump) | 01 (https://t.me/GMGN_sol_bot?start=7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump) | 02 (https://t.me/GMGN_sol02_bot?start=7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump) | 03 (https://t.me/GMGN_sol03_bot?start=7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump) | 04 (https://t.me/GMGN_sol04_bot?start=7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump)



Hitas.eth aka Alpha Hunter || 🥷 (https://gmgn.ai/sol/address/3XbzGrkkhSC8FQ5CPgTXGXTMTZZaBPmCuWCvFhmfVTnN) 4s ago
📈Cost $-- B/S:1/0
⏳Holding $1194.5642(5.9011 Sol) 

Skid (https://gmgn.ai/sol/address/HPkQu4osTzt3reYjYX8CHufGCGgHesc7PqG6WdhcY4p9) 1m ago
📈Cost $0.21971 B/S:9/0
⏳Holding $11.4K(56.2724 Sol) 

Dex (https://gmgn.ai/sol/address/mW4PZB45isHmnjGkLpJvjKBzVS5NXzTJ8UDyug4gTsM) 25m ago
📈Cost $0.4774 B/S:10/4
⏳Holding $0(-- Sol) 

jidn (https://gmgn.ai/sol/address/3h65MmPZksoKKyEpEjnWU2Yk2iYT5oZDNitGy5cTaxoE) 30m ago
📈Cost $0.12998 B/S:3/5
⏳Holding $0(-- Sol) 

Vincent.L /*acc | Alpha ᴺ (https://gmgn.ai/sol/address/3HmdSXaifCptP2vCveVXkUGoXUuwavzEvK8Bgb35Uwbb) 56m ago
📈Cost $0.24343 B/S:5/9
⏳Holding $0(-- Sol) 

浩哥 (https://gmgn.ai/sol/address/8wzb3sEeWRin4Jtnh15qrU43atgLnhDv26T1u4XUrtTi) 1h ago
📈Cost $0.30088 B/S:11/8
⏳Holding $7974.9065(39.3959 Sol) 

IN (https://gmgn.ai/sol/address/GaGbu7YACTn6PVTpZyyDeT3Cdypd2U3Z7pkSMxuz2FQw) 1h ago
📈Cost $0.3462 B/S:6/6
⏳Holding $0(-- Sol) 

冷静冷静再冷静 (https://gmgn.ai/sol/address/Ay9wnuZCRTceZJuRpGZnuwYZuWdsviM4cMiCwFoSQiPH) 1h ago
📈Cost $0.31252 B/S:43/34
⏳Holding $0(-- Sol) 

雪球💤🎶 (https://gmgn.ai/sol/address/77V4DGjRHzzW91gnJpKnVZ3xZXYezJmu7CgNb8GAkszH) 1h ago
📈Cost $0.32527 B/S:10/8
⏳Holding $0(-- Sol) 

Laolu🧙‍♂️ (https://gmgn.ai/sol/address/4625pi6asX1CpKmw1apw3mnGxxntS7F95ALqhWLgVoCr) 1h ago
📈Cost $0.1811 B/S:1/0
⏳Holding $4947.4502(24.4403 Sol) 

More (https://gmgn.ai/sol/token/7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump?wallet=3XbzGrkkhSC8FQ5CPgTXGXTMTZZaBPmCuWCvFhmfVTnN)

🏆 TIP: Add GMGN Bot to your group and earn commissions! 💸 Add bot (https://t.me/Alert_GMGNBOT)
"""

In [48]:
import re

def extract_keywords(text):
    data = {}

    # Patterns for extracting values
    patterns = {
        "num_kol_buy": r"(\d+)\s*KOL Buy",  # Extracts only the number
        "token_name": r"\$\w+\(([^)]+)\)", # sticker name
        "ca": r"\n([A-Za-z0-9]{42,46})",  # Matches the full token address
        "market_cap": r"MCP:\s*\$?([\w+\.\,]+)",  # Extracts MCP value with suffix
        "liq": r"Liq:\s*[\d\.\,]+\s*SOL\s*\(\$([\d\.KMB]+)",  # Extracts liquidity in $
        "top_10": r"TOP 10:\s*(\d+\.\d+%)"  # Extracts TOP 10 value as percentage
    }

    # Extract values using regex
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            value = match.groups()
            if key == "market_cap":  
                data[key] = string_to_float(value[0])  # Convert MCP
            elif key == "liq":
                data[key] = string_to_float(value[0])  # Convert Liquidity
            elif key == "top_10":
                data[key] = float(value[0].replace("%", ""))  # Convert % to float
            else:
                data[key] = int(value[0]) if key == "num_kol_buy" else value[0]
        else:
            data[key] = None

    return data

def string_to_float(s):
    multipliers = {
        'k': 1_000,
        'm': 1_000_000,
        'b': 1_000_000_000,
        't': 1_000_000_000_000
    }
    # Get the last character to determine the multiplier
    suffix = str(s)[-1]
    if suffix.isdigit():
        # No multiplier suffix; directly convert to float
        return float(s)
    else:
        # Extract the numeric part and apply the multiplier
        try:
            num = float(s[:-1])
            multiplier = multipliers.get(suffix.lower(), 1)
            return num * multiplier
        except ValueError:
            raise ValueError(f"Cannot convert '{s}' to float.")

# Extract and print the values
extracted_data = extract_keywords(text)
for key, value in extracted_data.items():
    print(f"{key}: {value}")


num_kol_buy: 88
token_name: Central African Republic Meme
ca: 7oBYdEhV4GkXC19ZfgAvXpJWp2Rn9pm1Bx2cVNxFpump
market_cap: 123800000.0
liq: 2200000.0
top_10: 7.03


In [49]:
from datetime import datetime
import pytz

def convert_unix_to_melbourne(unix_timestamp):
    # Define GMT and Melbourne timezones
    gmt_timezone = pytz.utc
    melbourne_timezone = pytz.timezone("Australia/Melbourne")

    # Convert Unix timestamp (GMT) to datetime object
    gmt_datetime = datetime.utcfromtimestamp(unix_timestamp).replace(tzinfo=gmt_timezone)

    # Convert to Melbourne timezone
    melbourne_datetime = gmt_datetime.astimezone(melbourne_timezone)

    return melbourne_datetime

# Example usage
unix_timestamp = 1739141550  # Example Unix timestamp
melbourne_time = convert_unix_to_melbourne(unix_timestamp)
print(melbourne_time)  # Output: Local Melbourne time



2025-02-10 09:52:30+11:00


In [50]:
from collections import defaultdict

# Sample data
tmp = [
    {
        'creator_name': 'alphashynobi',
        'creator_followers': 50524,
        'interactions_24h': 2688
    },
    {
        'creator_name': 'CapoXrpl',
        'creator_followers': 19306,
        'interactions_24h': 1394
    },
    {
        'creator_name': 'alphashynobi',
        'creator_followers': 50524,
        'interactions_24h': 3000
    }
]

# Use defaultdict to aggregate total interactions and followers per creator
totals = defaultdict(lambda: {'interactions_24h': 0, 'creator_followers': 0})

# Aggregate data
for post in tmp:
    name = post['creator_name']
    totals[name]['interactions_24h'] += post['interactions_24h']
    totals[name]['creator_followers'] = max(totals[name]['creator_followers'], post['creator_followers'])

# Display results
for name, data in totals.items():
    print(f"Creator: {name}, Total Interactions (24h): {data['interactions_24h']}, Total Followers: {data['creator_followers']}")


Creator: alphashynobi, Total Interactions (24h): 5688, Total Followers: 50524
Creator: CapoXrpl, Total Interactions (24h): 1394, Total Followers: 19306
